# Part 1 -----------------------------------------------

### Import libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


### Scrape data from the Wikipedia page and convert to a df

In [2]:
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(data, 'html.parser')
postalCodeList = []
boroughList = []
neighborhoodList = []
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        postalCodeList.append(cells[0].text.rstrip('\n'))
        boroughList.append(cells[1].text.rstrip('\n'))
        neighborhoodList.append(cells[2].text.rstrip('\n'))
toronto_df = pd.DataFrame({"PostalCode": postalCodeList,
                           "Borough": boroughList,
                           "Neighborhood": neighborhoodList})


In [3]:
toronto_df.head()

PostalCode           Borough               Neighborhood
0        M1A      Not assigned                           
1        M2A      Not assigned                           
2        M3A        North York                  Parkwoods
3        M4A        North York           Victoria Village
4        M5A  Downtown Toronto  Regent Park, Harbourfront

### Remove data with unassigned borough

In [8]:

toronto_df = toronto_df[toronto_df.Borough != 'Not assigned']
toronto_df.head()

PostalCode                            Neighborhood      Borough
0        M1B                          Malvern, Rouge  Scarborough
1        M1C  Rouge Hill, Port Union, Highland Creek  Scarborough
2        M1E       Guildwood, Morningside, West Hill  Scarborough
3        M1G                                  Woburn  Scarborough
4        M1H                               Cedarbrae  Scarborough

### Deal with multi neighborhood in the same postal code: Combine neighborhoods and add comma

In [5]:

toronto_df = toronto_df.groupby(["PostalCode", "Neighborhood"], as_index=False).agg(lambda x: ", ".join(x))
toronto_df.head()

PostalCode                            Neighborhood      Borough
0        M1B                          Malvern, Rouge  Scarborough
1        M1C  Rouge Hill, Port Union, Highland Creek  Scarborough
2        M1E       Guildwood, Morningside, West Hill  Scarborough
3        M1G                                  Woburn  Scarborough
4        M1H                               Cedarbrae  Scarborough

### Deal with neighborhood not assigned: assign them with borough

In [6]:

for index, row in toronto_df.iterrows():
    if row["Neighborhood"] == "Not assigned":
        row["Neighborhood"] = row["Borough"]
        
toronto_df.head()

PostalCode                            Neighborhood      Borough
0        M1B                          Malvern, Rouge  Scarborough
1        M1C  Rouge Hill, Port Union, Highland Creek  Scarborough
2        M1E       Guildwood, Morningside, West Hill  Scarborough
3        M1G                                  Woburn  Scarborough
4        M1H                               Cedarbrae  Scarborough

### Print the shape

In [7]:
toronto_df.shape

(103, 3)

# Part 2 -------------------------------------------------

### Load the csv file for postal-coordinate conversion

In [13]:
coordinates = pd.read_csv("https://cocl.us/Geospatial_data")
coordinates.rename(columns={"Postal Code": "PostalCode"}, inplace=True)
coordinates.head()

PostalCode   Latitude  Longitude
0        M1B  43.806686 -79.194353
1        M1C  43.784535 -79.160497
2        M1E  43.763573 -79.188711
3        M1G  43.770992 -79.216917
4        M1H  43.773136 -79.239476

### Merge two df

In [14]:
toronto_df = toronto_df.merge(coordinates, on="PostalCode")
toronto_df.head()

PostalCode                            Neighborhood      Borough   Latitude  \
0        M1B                          Malvern, Rouge  Scarborough  43.806686   
1        M1C  Rouge Hill, Port Union, Highland Creek  Scarborough  43.784535   
2        M1E       Guildwood, Morningside, West Hill  Scarborough  43.763573   
3        M1G                                  Woburn  Scarborough  43.770992   
4        M1H                               Cedarbrae  Scarborough  43.773136   

   Longitude  
0 -79.194353  
1 -79.160497  
2 -79.188711  
3 -79.216917  
4 -79.239476

# Part 3 -------------------------------------------------

### Determine the coordinate of Toronto

In [17]:
address = 'Toronto'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

43.6534817 -79.3839347


### Visualization

In [19]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Borough'], toronto_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

### Foursquare API

In [20]:
CLIENT_ID = 'VCN3U5CPYQU0ALTMZX4H313XLDOD2PNHZK4XRFHAUG0UDWCN' # your Foursquare ID
CLIENT_SECRET = 'VF25YVV3GMV1A3IPYA5RYQWSJI2RQXIP0K0EEVCKLGYFPQ1W' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: VCN3U5CPYQU0ALTMZX4H313XLDOD2PNHZK4XRFHAUG0UDWCN
CLIENT_SECRET:VF25YVV3GMV1A3IPYA5RYQWSJI2RQXIP0K0EEVCKLGYFPQ1W


In [21]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

In [27]:
venues = []

for lat, long, post, borough, neighborhood in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['PostalCode'], toronto_df['Borough'], toronto_df['Neighborhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            post, 
            borough,
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [30]:

# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['PostalCode', 'Borough', 'Neighborhood', 'BoroughLatitude', 'BoroughLongitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(2122, 9)


PostalCode      Borough                            Neighborhood  \
0        M1B  Scarborough                          Malvern, Rouge   
1        M1C  Scarborough  Rouge Hill, Port Union, Highland Creek   
2        M1C  Scarborough  Rouge Hill, Port Union, Highland Creek   
3        M1E  Scarborough       Guildwood, Morningside, West Hill   
4        M1E  Scarborough       Guildwood, Morningside, West Hill   

   BoroughLatitude  BoroughLongitude                  VenueName  \
0        43.806686        -79.194353                    Wendy’s   
1        43.784535        -79.160497      Royal Canadian Legion   
2        43.784535        -79.160497  Affordable Toronto Movers   
3        43.763573        -79.188711             RBC Royal Bank   
4        43.763573        -79.188711          G & G Electronics   

   VenueLatitude  VenueLongitude         VenueCategory  
0      43.807448      -79.199056  Fast Food Restaurant  
1      43.782533      -79.163085                   Bar  
2      43.787919      -79.162977         Moving Target  
3      43.766790      -79.191151                  Bank  
4      43.765309      -79.191537     Electronics Store

### Analyze each area

In [31]:
# one hot encoding
toronto_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add postal, borough and neighborhood column back to dataframe
toronto_onehot['PostalCode'] = venues_df['PostalCode'] 
toronto_onehot['Borough'] = venues_df['Borough'] 
toronto_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move postal, borough and neighborhood column to the first column
fixed_columns = list(toronto_onehot.columns[-3:]) + list(toronto_onehot.columns[:-3])
toronto_onehot = toronto_onehot[fixed_columns]

print(toronto_onehot.shape)
toronto_onehot.head()

(2122, 269)


PostalCode      Borough                           Neighborhoods  \
0        M1B  Scarborough                          Malvern, Rouge   
1        M1C  Scarborough  Rouge Hill, Port Union, Highland Creek   
2        M1C  Scarborough  Rouge Hill, Port Union, Highland Creek   
3        M1E  Scarborough       Guildwood, Morningside, West Hill   
4        M1E  Scarborough       Guildwood, Morningside, West Hill   

   Accessories Store  Afghan Restaurant  Airport  Airport Food Court  \
0                  0                  0        0                   0   
1                  0                  0        0                   0   
2                  0                  0        0                   0   
3                  0                  0        0                   0   
4                  0                  0        0                   0   

   Airport Gate  Airport Lounge  Airport Service  Airport Terminal  \
0             0               0                0                 0   
1             0               0                0                 0   
2             0               0                0                 0   
3             0               0                0                 0   
4             0               0                0                 0   

   American Restaurant  Antique Shop  Aquarium  Art Gallery  Art Museum  \
0                    0             0         0            0           0   
1                    0             0         0            0           0   
2                    0             0         0            0           0   
3                    0             0         0            0           0   
4                    0             0         0            0           0   

   Arts & Crafts Store  Asian Restaurant  Athletics & Sports  Auto Workshop  \
0                    0                 0                   0              0   
1                    0                 0                   0              0   
2                    0                 0                   0              0   
3                    0                 0                   0              0   
4                    0                 0                   0              0   

   BBQ Joint  Baby Store  Bagel Shop  Bakery  Bank  Bar  Baseball Field  \
0          0           0           0       0     0    0               0   
1          0           0           0       0     0    1               0   
2          0           0           0       0     0    0               0   
3          0           0           0       0     1    0               0   
4          0           0           0       0     0    0               0   

   Baseball Stadium  Basketball Stadium  Beach  Bed & Breakfast  Beer Bar  \
0                 0                   0      0                0         0   
1                 0                   0      0                0         0   
2                 0                   0      0                0         0   
3                 0                   0      0                0         0   
4                 0                   0      0                0         0   

   Beer Store  Belgian Restaurant  Bike Shop  Bistro  Boat or Ferry  \
0           0                   0          0       0              0   
1           0                   0          0       0              0   
2           0                   0          0       0              0   
3           0                   0          0       0              0   
4           0                   0          0       0              0   

   Bookstore  Boutique  Brazilian Restaurant  Breakfast Spot  Brewery  \
0          0         0                     0               0        0   
1          0         0                     0               0        0   
2          0         0                     0               0        0   
3          0         0                     0               0        0   
4          0         0                     0               0        0   

   Bridal Shop  Bubble Tea 

In [33]:
toronto_grouped = toronto_onehot.groupby(["PostalCode", "Borough", "Neighborhoods"]).mean().reset_index()

print(toronto_grouped.shape)
toronto_grouped.head()

(99, 269)


PostalCode      Borough                           Neighborhoods  \
0        M1B  Scarborough                          Malvern, Rouge   
1        M1C  Scarborough  Rouge Hill, Port Union, Highland Creek   
2        M1E  Scarborough       Guildwood, Morningside, West Hill   
3        M1G  Scarborough                                  Woburn   
4        M1H  Scarborough                               Cedarbrae   

   Accessories Store  Afghan Restaurant  Airport  Airport Food Court  \
0                0.0                0.0      0.0                 0.0   
1                0.0                0.0      0.0                 0.0   
2                0.0                0.0      0.0                 0.0   
3                0.0                0.0      0.0                 0.0   
4                0.0                0.0      0.0                 0.0   

   Airport Gate  Airport Lounge  Airport Service  Airport Terminal  \
0           0.0             0.0              0.0               0.0   
1           0.0             0.0              0.0               0.0   
2           0.0             0.0              0.0               0.0   
3           0.0             0.0              0.0               0.0   
4           0.0             0.0              0.0               0.0   

   American Restaurant  Antique Shop  Aquarium  Art Gallery  Art Museum  \
0                  0.0           0.0       0.0          0.0         0.0   
1                  0.0           0.0       0.0          0.0         0.0   
2                  0.0           0.0       0.0          0.0         0.0   
3                  0.0           0.0       0.0          0.0         0.0   
4                  0.0           0.0       0.0          0.0         0.0   

   Arts & Crafts Store  Asian Restaurant  Athletics & Sports  Auto Workshop  \
0                  0.0               0.0            0.000000            0.0   
1                  0.0               0.0            0.000000            0.0   
2                  0.0               0.0            0.000000            0.0   
3                  0.0               0.0            0.000000            0.0   
4                  0.0               0.0            0.111111            0.0   

   BBQ Joint  Baby Store  Bagel Shop    Bakery      Bank  Bar  Baseball Field  \
0        0.0         0.0         0.0  0.000000  0.000000  0.0             0.0   
1        0.0         0.0         0.0  0.000000  0.000000  0.5             0.0   
2        0.0         0.0         0.0  0.000000  0.142857  0.0             0.0   
3        0.0         0.0         0.0  0.000000  0.000000  0.0             0.0   
4        0.0         0.0         0.0  0.111111  0.111111  0.0             0.0   

   Baseball Stadium  Basketball Stadium  Beach  Bed & Breakfast  Beer Bar  \
0               0.0                 0.0    0.0              0.0       0.0   
1               0.0                 0.0    0.0              0.0       0.0   
2               0.0                 0.0    0.0              0.0       0.0   
3               0.0                 0.0    0.0              0.0       0.0   
4               0.0                 0.0    0.0              0.0       0.0   

   Beer Store  Belgian Restaurant  Bike Shop  Bistro  Boat or Ferry  \
0         0.0                 0.0        0.0     0.0            0.0   
1         0.0                 0.0        0.0     0.0            0.0   
2         0.0                 0.0        0.0     0.0            0.0   
3         0.0                 0.0        0.0     0.0            0.0   
4         0.0                 0.0        0.0     0.0            0.0   

   Bookstore  Boutique  Brazilian Restaurant  Breakfast Spot  Brewery  \
0        0.0       0.0                   0.0        0.000000      0.0   
1        0.0       0.0                   0.0        0.000000      0.0   
2        0.0       0.0                   0.0        0.142857      0.0   
3        0.0       0.0                   0.0        0.000000      0.0   
4        0.0       0.0                   0.0        0.000000     

In [34]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
areaColumns = ['PostalCode', 'Borough', 'Neighborhoods']
freqColumns = []
for ind in np.arange(num_top_venues):
    try:
        freqColumns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        freqColumns.append('{}th Most Common Venue'.format(ind+1))
columns = areaColumns+freqColumns

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['PostalCode'] = toronto_grouped['PostalCode']
neighborhoods_venues_sorted['Borough'] = toronto_grouped['Borough']
neighborhoods_venues_sorted['Neighborhoods'] = toronto_grouped['Neighborhoods']

for ind in np.arange(toronto_grouped.shape[0]):
    row_categories = toronto_grouped.iloc[ind, :].iloc[3:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    neighborhoods_venues_sorted.iloc[ind, 3:] = row_categories_sorted.index.values[0:num_top_venues]

# neighborhoods_venues_sorted.sort_values(freqColumns, inplace=True)
print(neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted

(99, 13)


PostalCode           Borough  \
0         M1B       Scarborough   
1         M1C       Scarborough   
2         M1E       Scarborough   
3         M1G       Scarborough   
4         M1H       Scarborough   
5         M1J       Scarborough   
6         M1K       Scarborough   
7         M1L       Scarborough   
8         M1M       Scarborough   
9         M1N       Scarborough   
10        M1P       Scarborough   
11        M1R       Scarborough   
12        M1S       Scarborough   
13        M1T       Scarborough   
14        M1V       Scarborough   
15        M1W       Scarborough   
16        M2H        North York   
17        M2J        North York   
18        M2K        North York   
19        M2L        North York   
20        M2N        North York   
21        M2P        North York   
22        M2R        North York   
23        M3A        North York   
24        M3B        North York   
25        M3C        North York   
26        M3H        North York   
27        M3J        North York   
28        M3K        North York   
29        M3L        North York   
30        M3M        North York   
31        M3N        North York   
32        M4A        North York   
33        M4B         East York   
34        M4C         East York   
35        M4E      East Toronto   
36        M4G         East York   
37        M4H         East York   
38        M4J         East York   
39        M4K      East Toronto   
40        M4L      East Toronto   
41        M4M      East Toronto   
42        M4N   Central Toronto   
43        M4P   Central Toronto   
44        M4R   Central Toronto   
45        M4S   Central Toronto   
46        M4T   Central Toronto   
47        M4V   Central Toronto   
48        M4W  Downtown Toronto   
49        M4X  Downtown Toronto   
50        M4Y  Downtown Toronto   
51        M5A  Downtown Toronto   
52        M5B  Downtown Toronto   
53        M5C  Downtown Toronto   
54        M5E  Downtown Toronto   
55        M5G  Downtown Toronto   
56        M5H  Downtown Toronto   
57        M5J  Downtown Toronto   
58        M5K  Downtown Toronto   
59        M5L  Downtown Toronto   
60        M5M        North York   
61        M5N   Central Toronto   
62        M5P   Central Toronto   
63        M5R   Central Toronto   
64        M5S  Downtown Toronto   
65        M5T  Downtown Toronto   
66        M5V  Downtown Toronto   
67        M5W  Downtown Toronto   
68        M5X  Downtown Toronto   
69        M6A        North York   
70        M6B        North York   
71        M6C              York   
72        M6E              York   
73        M6G  Downtown Toronto   
74        M6H      West Toronto   
75        M6J      West Toronto   
76        M6K      West Toronto   
77        M6L        North York   
78        M6M              York   
79        M6N              York   
80        M6P      West Toronto   
81        M6R      West Toronto   
82        M6S      West Toronto   
83        M7A  Downtown Toronto   
84        M7R       Mississauga   
85        M7Y      East Toronto   
86        M8V         Etobicoke   
87        M8W         Etobicoke   
88        M8X         Etobicoke   
89        M8Y         Etobicoke   
90        M8Z         Etobicoke   
91        M9B         Etobicoke   
92        M9C         Etobicoke   
93        M9M        North York   
94        M9N              York   
95        M9P         Etobicoke   
96        M9R         Etobicoke   
97        M9V         Etobicoke   
98        M9W         Etobicoke   

                                        Neighborhoods  \
0                                      Malvern, Rouge   
1              Rouge Hill, Port Union, Highland Creek   
2                   Guildwood, Morningside, West Hill   
3                                              Woburn   
4                                           Cedarbrae   
5                                 Scarborough Village   
6         Kennedy Park, Ionview, East Birchmount Park   
7                     Golden Mile, Clairlea, O